# Check which Lipids are ACM dependent
- Compare lipids found in Supernatant and Cells depending whether ACM as used or not

### Included libraries

In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
from statsmodels.stats.multitest import multipletests
from matplotlib import cm
from matplotlib import pylab as plt
from matplotlib.lines import Line2D
import seaborn as sns

### Functions and definitions

In [2]:
#define classes of lipids e.g. PC = Phosphatidylcholines
types_of_Lipids = ['CE','Cer','DAG','LPC','LPE','PC','PE','PI','PS','SM','TAG']

#colormap (20 unique colors)
cmap = cm.get_cmap('tab20')

#assign for each class of lipid a unique color
lipid_color = {}
for i,l in enumerate(types_of_Lipids):
    lipid_color[l] = cmap(i)


### Main Code

In [3]:
#Load the actual lipid results
LipidData = pd.read_excel('../data/Report_MEC025_LIPID01_jb.xlsx' ,header=2, na_values='<LOD')

In [5]:
#extract the lipids
columns = LipidData.columns
Lipids = columns[7:]
print (Lipids)

Index(['CE 16:0', 'CE 16:1', 'CE 18:0', 'CE 18:1', 'CE 18:2', 'CE 18:3',
       'CE 20:0', 'CE 20:3', 'CE 20:4', 'CE 20:5',
       ...
       'TAG 54:5', 'TAG 54:6', 'TAG 55:1', 'TAG 55:3', 'TAG 56:1', 'TAG 56:4',
       'TAG 56:7', 'TAG 57:1', 'TAG 58:8', 'TAG 58:9'],
      dtype='object', length=283)


In [10]:
#make analysis both for the cells and the supernatant experiment (where lipids are found)
specimens = ['cells','supernatant']

#make both analysis
for spec in specimens:
    
    #extract WT control and WT with addition of ACM 
    data = LipidData.loc[(LipidData['Specimen'] == spec) & ((LipidData['Experiment'] == 'WT_C') | (LipidData['Experiment'] == 'WT_ACM'))]
    
    #remove entries that have no values
    data = data.dropna(axis=1,how='all')
    
    #remaining lipids contains all valid columns (=lipids)
    remaining_Lipids = data.columns.values[7:]
    
    # The next 4 variables contain the (statistical )results for the individiual lipids (comparisan WT control and ACM)
    # foldchanges = foldchanges 
    # significance = student_ttest
    # calculated_lipids = name of lipid
    # color = color for lipidclass (see before)
    ####
    foldchanges = []
    significance = []
    calculated_lipids = []
    color = []
    # go through all valid lipids
    for Lipid in remaining_Lipids:
        
        # extract results for control and ACM
        WT_C_values =  data.loc[data['Experiment'] == 'WT_C'][Lipid]
        WT_ACM_values =  data.loc[data['Experiment'] == 'WT_ACM'][Lipid]
        
        #drop rows with no values 
        WT_C_values = WT_C_values.dropna()
        WT_ACM_values = WT_ACM_values.dropna()
        
        #only calculate statistics if valid results found for both groups
        if len(WT_C_values) > 0 and len(WT_ACM_values) > 0:
            
            #calculate the statistics (see before)
            significance.append(ttest_ind(WT_C_values,WT_ACM_values)[1])
            foldchanges.append(WT_ACM_values.mean()/WT_C_values.mean())
            calculated_lipids.append(Lipid)
            color.append(lipid_color[Lipid.split(' ')[0]])
    

    # correct pValues according to benjamini hochberg (FDR)
    pValues_Corrected =  multipletests(significance,alpha=0.05,method='fdr_bh')[1]
    pValues_Corrected = [-np.log10(p) for p in pValues_Corrected]
    
    #transform the foldchanges to log2
    foldchanges = [np.log2(f) for f in foldchanges]
    
    ####
    # START MAKING PLOT ACM DEPENDENCY WT
    # Scatter plot showing for each individual lipid the foldchange (x-axis) and pvalue (y-axis)
    ####
    
    # Make result plot for differences upon ACM treatment (for wildtype)
    plt.title('WT_ACM / WT_C - ' + spec)
    plt.scatter(foldchanges,pValues_Corrected,c=color, alpha = 0.4)
    plt.axhline(-np.log10(0.05), color= 'grey', ls='--')
    plt.xlabel('log2[Fold Change ACM/Control]')
    plt.ylabel('-log[PValue]')
    
    # Make a legend showing the colors to lipid classes
    legend_elements = []
    for key in lipid_color:
        legend_elements.append(Line2D([0], [0], marker='o', color='w', label=key,
                          markerfacecolor=lipid_color[key], markersize=10))

    # Make actual plot
    plt.legend(handles=legend_elements, loc='upper right',prop={'size': 5})
    plt.savefig('../results/Lipid_ACM_Dependency/ACM_Dependency_'+spec+'.pdf')
    plt.close()
    
    ####
    # END PLOT ACM DEPENDENCY WT
    ####
    
    
    
    
    ###
    # MAKE PLOT ALL LIPIDS FOLDCHANGE
    ####
    
    # sort the foldchanges for the individual lipids (from largest to smalest)
    foldchanges_sort, calculated_lipids_sort  = zip(*sorted(zip(foldchanges, calculated_lipids),reverse=True))
    foldchanges_sort, colors_sort  = zip(*sorted(zip(foldchanges, color),reverse=True))
    foldchanges_sort = [[x] for x in foldchanges_sort]

    # Make plot showing the individual lipid foldchanges (largest top, smallest bottom)
    # No error bar (as it consists already of too many individual bars)
    plt.title('WT_ACM / WT_C - ' + spec)
    sns.barplot(data=foldchanges_sort,orient='h',palette=colors_sort,linewidth=None)
    plt.legend(handles=legend_elements, loc='right',prop={'size': 5})
    plt.xlabel('log2[Fold Change ACM/Control]')
    plt.yticks(range(0,len(calculated_lipids_sort)),calculated_lipids_sort,rotation=0,size=1.8)
    plt.savefig('../results/Lipid_ACM_Dependency/ACM_Dependency_Barplot_AllLipids_'+spec+'.pdf')
    plt.close()
    
    ###
    # END PLOT ALL LIPIDS FOLDCHANGE
    ####
    
    
    
    ###
    # MAKE PLOT LIPID GROUPs FOLDCHANGE
    ####
    
    # Create dictionary for the individual lipid groups
    Lipid_Group_Results = {}
    for l in types_of_Lipids:
        Lipid_Group_Results[l] = []
    
    #add the individual lipids to the previosuly defined dictionary
    for fc,lipid in zip(foldchanges,calculated_lipids):
        Lipid_Group_Results[lipid.split(' ')[0]].append(fc)
        
    
    
    # result lists that will contain the results for the whole lipid grups (same as before)
    Lipid_Groups = []
    Foldchanges_Groups = []
    Foldchanges_Groups_Means = []
    Colors_Groups = []
    #go through all lipid groups
    for key in Lipid_Group_Results:
        #calculate only results if the lipid groups contains 1 or more lipids
        if len(Lipid_Group_Results[key]) != 0:
            Lipid_Groups.append(key)
            Foldchanges_Groups.append(Lipid_Group_Results[key])
            Foldchanges_Groups_Means.append(np.mean(Lipid_Group_Results[key]))
            Colors_Groups.append(lipid_color[key])
      
    #sort the foldchanges again from largest to smallest
    _, Foldchanges_Groups  = zip(*sorted(zip(Foldchanges_Groups_Means, Foldchanges_Groups),reverse=True))
    _, Colors_Groups  = zip(*sorted(zip(Foldchanges_Groups_Means, Colors_Groups),reverse=True))
    _, Lipid_Groups  = zip(*sorted(zip(Foldchanges_Groups_Means, Lipid_Groups),reverse=True))
    
    #make bar plot with SEM as error bar 
    plt.title('WT_ACM / WT_C - ' + spec)
    sns.barplot(data=Foldchanges_Groups,orient='h',palette=Colors_Groups, ci=68, capsize=0.3, errwidth=0.7,errcolor='black', linewidth=1, edgecolor='black')
    plt.legend(handles=legend_elements, loc='right',prop={'size': 5})
    plt.xlabel('log2[Fold Change ACM/Control] (+/- SEM)')
    plt.yticks(range(0,len(Lipid_Groups)),Lipid_Groups,rotation=0,size=10)
    plt.savefig('../results/Lipid_ACM_Dependency/ACM_Dependency_Barplot_LipidGroups_'+spec+'.pdf')
    plt.close()
    
    ###
    # END PLOT LIPID GROUPs FOLDCHANGE
    ####
    
    